In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

In [30]:
df = pd.read_csv('Data/video_games_sales_completo.csv')

In [31]:
print(df.columns)

Index(['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Ventas_Clase',
       'Nota PEGI', 'User Score', 'User Ratings Count', 'Platforms Info',
       'Estado_Consola', 'Price', 'Price_Platform', 'Year_Consola',
       'Play Time', 'Modo Juego', 'Años_desde_lanzamiento_consola',
       'Precio_relativo', 'PEGI_categoria', 'Duracion_juego_cat',
       'Nombre_Base', 'Es_Saga', 'Tipo_Saga'],
      dtype='object')


In [32]:
# Definicion de variables
X = df.drop(columns=['Ventas_Clase', 'Global_Sales', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Name', 'User Ratings Count', 'Platforms Info', 'Nota PEGI','Play Time','Nombre_Base', 'Es_Saga'])  # Variables independientes
y = df['Ventas_Clase']                                  # Variable dependiente

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Definir las columnas categóricas para One-Hot Encoding
categorical_columns = ['Platform', 'Year', 'Genre', 'Publisher', 'PEGI_categoria', 'Estado_Consola', 'Year_Consola', 'Modo Juego', 'Duracion_juego_cat','Tipo_Saga']

# Crear un transformador para las columnas categóricas (One-Hot Encoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_columns),  # One-Hot Encoding en las columnas categóricas
        ('num', StandardScaler(), X.select_dtypes(include=['float64', 'int64']).columns)  # Escalado en columnas numéricas
    ])

# Crear el pipeline que primero realiza la transformación y luego aplica el modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=300))
])

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Hacer predicciones
y_pred = pipeline.predict(X_test)

# Evaluar el modelo
print(f"Exactitud: {accuracy_score(y_test, y_pred)}")
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Exactitud: 0.5096350963509635
Matriz de confusión:
[[ 311  192   94  382]
 [  89 1431   26  392]
 [ 173   48  152  117]
 [ 252  571   56  592]]
Reporte de clasificación:
              precision    recall  f1-score   support

        Alta       0.38      0.32      0.34       979
        Baja       0.64      0.74      0.68      1938
  Gran Éxito       0.46      0.31      0.37       490
       Media       0.40      0.40      0.40      1471

    accuracy                           0.51      4878
   macro avg       0.47      0.44      0.45      4878
weighted avg       0.50      0.51      0.50      4878



c:\Users\pablo\miniconda3\envs\proyecto_final\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [33]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy del modelo: {accuracy:.2f}")

Accuracy del modelo: 0.51
